# Capstone Project - The Battle of the Neighborhoods (Week 2)
### Applied Data Science Capstone by IBM/Coursera

## Table of contents
* [Introduction: Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)



## Introduction: Problem <a name="introduction"></a>

In this project, we will explore the schools in a neighborhood by comparing them based on various features. Specifically, this exercise is to group similar schools in **San Mateo county in California.**

We will use various features such as **grade level, gender, tests performance, economic criteria, ethnicity of student and parent education** etc., to create the school clusters. There are many different choices of selecting certain combinations of attributes, this grouping exercise of schools may help in identifying the *influencing criteria* of such characteristics.

This study would help in making certain decisions based on individual’s choice, such as moving to San Mateo county in California, for either easier commute or may have more affordable housing or ethnic culture or for any other personal and/or business reasons.

## Data <a name="data"></a>

**We need the following data items, based on the above problem description:**

- List of schools and locations in county of San Mateo, California
- Tests performance data of the schools for different grades, gender, ethnicity and various other attributes 
- The geolocation data i.e. Latitude, Longitude for each school in San Mateo

**The following data sources will be needed to extract the required information:**

1. The **“Research Files”** at <a href="https://caaspp.cde.ca.gov/sb2018/ResearchFileList" >California Department of Education</a> for Smarter Balanced Assessments data can be downloaded as csv files. These research files contain results from the administrations of the California Assessment of Student Performance and Progress (CAASPP) Smarter Balanced Assessments. As per the website, these research files require two primary tables, the *entities* and the *test data*.
2. **Entities:** lists the County, District, and School entity name, code and zip-code for all entities as the existed in the administration year selected. This file must be merged with the research file to join these entity names with the appropriate score data.
3. **Tests data:** comprised of the school, district, county, and state aggregate CAASPP counts and scores.
4. **Other supporting data:** can be downloaded from the above website, the *Student Group ID,* identifies each demographic student group and ID reported in the CAASPP results. The *Test ID,* each student will take a number of tests and a specific test should be selected during analysis. 
5. **Geocoding data:** obtain the school location data i.e. Latitude, Longitude based on the name of the school and zip-code, using Google Maps API geocoding.

### Entities - California Schools data

Entities file has data for Counties, Districts and Schools. It has the names and codes of all these entities and their zip-codes for California state-wide schools.

![Sample California School Entities](Sample_CA_School_Entities.png "Sample California School Entities")

In this project, we are only interested in San Mateo County in California, which is "County Code = 41". Therefore, after loading the provided "sb_ca2018entities.csv" file, filter for San Mateo county. Next, delete the rows with no School Name (these are of Type ID -> 4=State, 5=County, 6=District), and we need only Schools. The result is a dataset of 197 rows (schools) in San Mateo County.


In [1]:
# Load the California School Entities file that has the information about the schools in California.

import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

df_schools = pd.read_csv('sb_ca2018entities.csv')

df_schools.head()

,County Code,District Code,School Code,Filler,Test Year,Type Id,County Name,District Name,School Name,Zip Code
0,0,0,0,NaN,2018,4,State of California,NaN,NaN,
1,1,0,0,NaN,2018,5,Alameda,NaN,NaN,
2,1,10017,0,NaN,2018,6,Alameda,Alameda County Office Of Education,NaN,
3,1,10017,112607,NaN,2018,9,Alameda,Envision Academy For Arts & Technology,Envision Academy For Arts & Technology,94612
4,1,10017,123968,NaN,2018,9,Alameda,Community School For Creative Education,Community School For Creative Education,94606


In [2]:
# Extract San Mateo, CA county, with County Code = 41, there are 10 different columns, including Filler column
# Drop the rows with empty School Name and reset index

df_schools_sm = pd.DataFrame(df_schools[df_schools['County Code'] == 41])

# Drop the rows with empty School Name
df_schools_sm.dropna(subset=["School Name"], axis=0, inplace=True)

# reset index, because we dropped rows
df_schools_sm.reset_index(drop=True, inplace=True)

df_schools_sm.shape

(197, 10)

In [3]:
# Drop "Filler" column and keep the required columns such as "County Code","District Code","School Code",
# "Test Year","Type Id","County Name","District Name","School Name","Zip Code" => Now, 9 different columns

df_schools_sm.drop(['Filler'], axis = 1, inplace = True, errors = 'ignore')

# Drop rows with null or empty values in any of these 9 columns
df_schools_sm.dropna(inplace = True)

# reset index, because we dropped column
df_schools_sm.reset_index(drop=True, inplace=True)

df_schools_sm.shape  # should have 9 columns

(197, 9)

### Geocoding data for San Mateo Schools

Let's find the latitude & longitude for San Mateo Schools, using Google Maps geocoding API. Unfortunately, Foursquare did not have this location data. We can't just use the School Name since the School Name may be repeated in different districts, such as "Hoover Elementary" in "Burlingame Elementary" district and also in "Redwood City Elementary" district, however, they have different zip-codes. 

Therefore, for Google Maps geocoding API, use the combination of School Name and Zip-Code to uniquely identify a school, to avoid the above described issue. Alternatively, use the provided "SM_Schools_Geospatial_Cordinates.csv" file, has the School Code, Latitude and Longitude for the 197 schools in San Mateo county.

Now, merge the schools data frame and the geospatial data frame on "School Code" so that we have the Latitude and Longitude for every school in San Mateo county.

In [4]:
! pip install -U googlemaps

Requirement already up-to-date: googlemaps in /Users/ravibhavanasi/anaconda3/lib/python3.7/site-packages (4.2.0)


In [5]:
# Use your Google API Key

google_api_key = 'YOUR_GOOGLE_API_KEY'

In [6]:
# The function get_coordinates returns lattitude and longtitude, uses the Google Maps to get the lattitue 
# and Longitude for a given address. if for some reason the call to google fails, then returns None, None

import googlemaps

import requests

def get_coordinates(api_key, address, verbose=False):
    try:
        url = 'https://maps.googleapis.com/maps/api/geocode/json?key={}&address={}'.format(api_key, address)
        response = requests.get(url).json()
        if verbose:
            print('Google Maps API JSON result =>', response)
        results = response['results']
        geographical_data = results[0]['geometry']['location'] # get geographical coordinates
        lat = geographical_data['lat']
        lon = geographical_data['lng']
        return [lat, lon]
    except:
        print('Something went wrong getting lat, long for address {} Google Maps API JSON result => {}'.format(address, response))
        return [None, None]

In [7]:
# Get the school code, name and zip code
# Some schools names that are repeated but with different zip codes, combine and School Name and 
# Zip code as address to pass it to Google maps to find the lat and long

unique_schools = df_schools_sm[["School Code","School Name","Zip Code"]]

schools_lat_long = [] # Create a list of school codes with lat and long


try:
    # Load the San Mateo Schools lat and long from the provided file
    schools_loc_data = pd.read_csv('SM_Schools_Geospatial_Cordinates.csv')
    
except:
    # if error while file loading for any reasons, get the locations by calling the Google Maps
    # Some schools names that are repeated but with different zip codes, combine and School Name and 
    # Zip code as address to pass it to Google maps to find the lat and long
    for i in unique_schools.itertuples():
        address = i[2]+", "+i[3]
        addr_cords = get_coordinates(google_api_key,str(address))   
        schools_lat_long.append({'School Code': i[1], 'Latitude': addr_cords[0], 'Longitude': addr_cords[1]})
    # make the dataframe from the list
    schools_loc_data = pd.DataFrame(schools_lat_long)

In [8]:
# Just to make sure there are 197 schools with School Code, Latitude and Longitude

schools_loc_data.shape

(197, 3)

In [9]:
### "No Cordinate Schools" ###

# While using the Google Maps API, for any reasons if the Google Maps fails to return valid Lat and Long, then 
# make a list of schools (school codes) that Google Maps couldn't return valid Lat and Long.  Unfortunately, 
# we can't modify the name of the school to make the Google Maps work, because School names are downloaded from 
# the CA Education website (mentioned above)

# Ideally, we want this to be empty list so that we have geospatial coordinates for all schools
# keep the list of "no cordinates" School Codes so that we can make a decision whether to remove these schools 
# from the School test data while doing the clustering or while ploting them on the map as these schools don't 
# have the valid lat & long.

no_cordinates = schools_loc_data[schools_loc_data['Latitude'].isnull()]
no_cord_schools_list = list(no_cordinates['School Code'])
no_cord_schools_list # Empty list is good, so that we have geospatial coordinates for all schools

[]

In [10]:
# Merge the location data to the schools data Data Frame on School Code

df_schools_sm = pd.merge(df_schools_sm, schools_loc_data, on='School Code')

df_schools_sm.dropna(inplace = True)

df_schools_sm.reset_index(drop=True, inplace = True)

df_schools_sm.shape # Now, we should have 11 columns (earlier 9 + Lat + Long = 11 columns)

(197, 11)

### Tests Data - San Mateo Schools Smarter Balanced tests performance data for 2018

The data file contains Smarter Balanced tests performance scores and stats for 2018 for each test and rows for each test, school, Grade and student subgroup etc., for San Mateo Schools. Students take multiple Smarter Balanced tests such as English Language Arts (ELA), Mathematics which have unique Test Ids. Each school administers tests for different grades that the school offers. The data at school level is represented by Grade Id 13. The file also contains rows of data for aggregates at School Total, District Total, County Total for each Test id. In order to protect student confidentiality, no scores are reported (or included in the research files) for any group of 10 or fewer students.

![Sample San Mateo Schools SB Tests Data](Sample_SM_Schools_SB_Tests_Data.png "Sample San Mateo Schools SB Tests Data")

After loading the provided "sb_ca2018_all_41_v3.csv" file, let's delete all the rows with School Code = 0 (these are Summary rows), and I chose to delete the rows (schools) where couldn't get the lat & long using the Google Maps API (as mentioend in the above **"Entities - California Schools data"** section under "No Coordinate Schools"). [Since I am using the provided geospatial coorindates file, all schools have the lat & long]

Also, filtered data to limit to 8th Grade and Math test to reduce the data size and get a representative sample. Additionally, kept only 12 columns of interest.

In [11]:
# Load the provided San Mateo Schools tests performance csv data file

df_data_sm = pd.read_csv('sb_ca2018_all_41_v3.csv') # For San Mateo (County Code = 41)

df_data_sm.head()

,County Code,District Code,School Code,Filler,Test Year,Subgroup ID,Test Type,Total Tested At Entity Level,Total Tested with Scores,Grade,Test Id,CAASPP Reported Enrollment,Students Tested,Mean Scale Score,Percentage Standard Exceeded,Percentage Standard Met,Percentage Standard Met and Above,Percentage Standard Nearly Met,Percentage Standard Not Met,Students with Scores,Area 1 Percentage Above Standard,Area 1 Percentage Near Standard,Area 1 Percentage Below Standard,Area 2 Percentage Above Standard,Area 2 Percentage Near Standard,Area 2 Percentage Below Standard,Area 3 Percentage Above Standard,Area 3 Percentage Near Standard,Area 3 Percentage Below Standard,Area 4 Percentage Above Standard,Area 4 Percentage Near Standard,Area 4 Percentage Below Standard
0,41,0,0,NaN,2018,1,B,48680,48586,3,2,6894,6810,2460.2,34.53,27.71,62.24,19.19,18.57,6806,47.30,30.22,22.49,39.96,38.44,21.60,41.65,40.46,17.89,0.00,0.00,0.00
1,41,0,0,NaN,2018,1,B,48406,48245,3,1,6894,6736,2447.1,35.77,22.82,58.59,20.93,20.47,6731,34.66,43.25,22.09,31.48,43.58,24.94,27.77,58.11,14.12,35.48,46.12,18.40
2,41,0,0,NaN,2018,1,B,48406,48245,4,1,7157,6999,2490.7,37.19,22.74,59.93,16.31,23.76,6996,34.70,44.44,20.86,33.85,42.55,23.60,27.41,59.58,13.01,34.52,46.43,19.05
3,41,0,0,NaN,2018,1,B,48680,48586,4,2,7157,7058,2498.7,31.17,26.27,57.45,24.50,18.05,7057,43.33,27.63,29.04,35.10,41.41,23.48,37.21,39.95,22.84,0.00,0.00,0.00
4,41,0,0,NaN,2018,1,B,48680,48586,5,2,7124,7041,2523.4,32.88,17.95,50.82,22.82,26.36,7038,39.33,26.14,34.53,32.45,39.29,28.26,30.09,42.61,27.30,0.00,0.00,0.00


In [12]:
# Extract schools tests data not in "no_cord_schools_list"
  
df_data_sm = df_data_sm[~df_data_sm['School Code'].isin(no_cord_schools_list)]

df_data_sm.shape

(63309, 32)

In [13]:
# Drop the rows with summary values, i.e. School Code = 0

df_data_sm.drop(df_data_sm[df_data_sm['School Code'] == 0].index,inplace = True)

# Reset the index after drop
df_data_sm.reset_index(drop=True, inplace = True)

df_data_sm.shape

(50609, 32)

In [14]:
#Test ID	Test ID Num	Test Name
#1	1	SB - English Language Arts/Literacy
#2	2	SB - Mathematics

df_data_sm.drop(df_data_sm[df_data_sm['Test Id'] == 1].index,inplace = True)

In [15]:
# Drop all records that are not grade 8

df_data_sm.drop(df_data_sm[df_data_sm['Grade'] != 8].index,inplace = True)

In [16]:
df_data = df_data_sm[['District Code', 'School Code', 'Subgroup ID', 'Grade', 'Test Id',
       'Students Tested', 'Mean Scale Score', 'Percentage Standard Exceeded',
       'Percentage Standard Met', 'Percentage Standard Met and Above',
       'Percentage Standard Nearly Met', 'Percentage Standard Not Met']]

df_data.shape # should have only 12 columns

(1984, 12)


Now, we have data of the Schools entities and the tests performance for each school, also the spatial coordinates for each school in San Mateo county in California.


## Methodology <a name="methodology"></a>

In this project, we will cluster the San Mateo schools based on the tests performance data and few selective Student Group attributes.

**Step 1:** We have collected the required **data: Schools, Tests Scores and location** for all schools in San Mateo county.

**Step 2:** We can now use this data and select various student sub groups (46 different subgroup Ids) for clustering of the schools, the following section has the details on Student Groups/Subgroups IDs. Our target is to get one row for each school with features (columns) as Students Tested, Mean Score and Percentages above and below Passing grades etc., with counts of students per Subgroups. We want to measure the impact of these numbers on total score, so do not need scores per Subgroup, only the count of students. While, there may be many different ways (methods) to select various sub groups and, I just picked the following sub group categories and respective sub group Ids, as per my personal interest:

* **Gender**
* **Economic Status**
* **Ethnicity**
* **Parent Education**


**Step 3:** In the final step, we will use the selected sub group attributes, normalize the data and use K-means clustering, and explore the clusters



### Student Group/Subgroups IDs

Data for each Student Group (Subgroup ID) is available as well as for all students tested. Subgroups identify sub-totals by Gender, Ethnicity, Economic status, English Language Fluency, Parents Education level, Immigration status etc., They are grouped in to 10 different categories and each category has different subgroups. For example, **"Ethnicity"** has Student Groups with different (Student Group ID) Subgroup Id, such as "Black or African American"=74; "American Indian or Alaska Native"=75; "Filipino"=77; "Asian"=76 etc., Note that "All Students" has the "Subgroup ID=1", see the sample below

![Sample Student Group Data](Sample_Student_Group_Data.png "Sample Student Group Data")

Let's get various subtotal records separated, and merge them into another dataframe for final analysis. Finally, use the provided "CAA_ca_Subgroups.csv" file for mapping the Subgroup ID columns to descriptive names.


In [17]:
# Transpose the Subgroup IDs to columns

# Get Number of students tested per subgroup exclusing the total tested (subgroup=1, which is all students)
df_data_subgroups = df_data.loc[df_data['Subgroup ID']!=1,['District Code', 'School Code', 'Grade', 'Test Id','Subgroup ID','Students Tested']]

# Transpose the Subgroup Ids as columns with Students Tested as the values under each corresponding column
df_subgroups_pivot = df_data_subgroups.pivot_table(index=['District Code', 'School Code', 'Grade', 'Test Id'], columns=['Subgroup ID'],values='Students Tested', aggfunc='first', fill_value=0)

df_subgroups_pivot.head()

# Reset the index

df_subgroups_pivot.reset_index()

Subgroup ID,District Code,School Code,Grade,Test Id,3,4,6,7,8,28,31,74,75,76,77,78,79,80,90,91,92,93,94,99,111,120,121,128,142,144,160,170,180,190,200,201,202,203,204,205,206,207,220,221,222,223,224,225,226,227
0,10413,4130076,8,2,*,*,*,0,*,0,*,*,*,0,0,*,0,*,*,*,*,*,0,*,*,0,0,*,*,0,*,*,*,0,*,*,0,0,*,0,*,0,0,0,0,0,*,0,0,0
1,10413,6069363,8,2,0,*,*,0,0,0,0,*,0,0,0,0,0,0,0,0,*,0,0,0,*,0,0,*,0,0,0,0,*,0,0,0,0,0,0,0,0,0,*,0,0,0,0,0,0,0
2,68858,6043392,8,2,28,21,38,*,21,0,32,*,0,*,12,22,*,*,11,11,14,*,*,45,17,*,*,*,*,*,11,32,13,0,*,0,*,*,18,*,*,0,*,0,*,*,*,0,0,*
3,68866,4168866,8,2,*,0,*,0,0,0,0,0,0,0,0,0,0,*,0,0,0,*,*,0,*,0,0,*,*,*,*,*,*,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,*,*
4,68866,6043475,8,2,22,16,34,*,*,0,13,*,0,*,*,12,*,12,*,*,*,12,*,27,25,0,*,11,*,*,*,11,23,*,*,0,*,0,*,*,*,*,*,0,*,*,*,*,11,*
5,68866,6043483,8,2,203,212,406,60,68,0,35,*,*,110,11,46,*,207,*,12,39,155,184,392,380,*,19,23,*,30,*,77,278,*,*,*,*,*,12,*,11,*,*,0,107,*,34,*,196,28
6,68874,6043509,8,2,25,28,50,*,*,0,12,0,0,13,*,11,0,21,*,*,13,21,11,49,41,0,0,*,*,*,*,12,33,0,0,0,*,*,*,0,*,0,0,0,*,*,*,0,20,*
7,68882,4168882,8,2,0,*,*,0,0,0,0,0,0,0,0,*,0,0,0,0,0,0,*,0,*,0,0,*,0,0,0,0,*,0,0,0,0,0,0,0,0,0,0,0,0,0,*,0,0,0
8,68882,6043525,8,2,173,172,334,*,81,0,42,*,*,78,13,43,0,175,*,16,30,107,175,318,303,*,*,27,*,25,11,92,248,0,0,0,*,*,16,0,13,0,*,*,70,*,27,0,162,25
9,68890,6043665,8,2,131,123,221,*,84,*,127,*,0,*,*,132,0,98,80,23,32,58,51,239,127,*,*,15,31,11,33,117,133,0,0,0,*,*,112,0,*,*,*,0,*,*,20,0,89,*


In [18]:
# Get "All students" (Subgroup Id = 1) from the performance data set

df_schools_1 = pd.DataFrame
df_schools_1 = df_data.loc[df_data['Subgroup ID']==1]

# Merge the performance data set with the subgroups as columns data frame 
df_schools_final = pd.DataFrame()
df_schools_final = pd.merge(df_schools_1, df_subgroups_pivot, on='School Code')

# Let's clean the data
import numpy as np
df_schools_final.replace("*",0,inplace = True) # get everything to numeric, * represents populations below 10 to protect student confidentiality

# Drop records with 0 Students Tested
df_schools_final.drop(df_schools_final[df_schools_final['Students Tested'] == 0].index,inplace = True)

df_schools_final.head(10)

,District Code,School Code,Subgroup ID,Grade,Test Id,Students Tested,Mean Scale Score,Percentage Standard Exceeded,Percentage Standard Met,Percentage Standard Met and Above,Percentage Standard Nearly Met,Percentage Standard Not Met,3,4,6,7,8,28,31,74,75,76,77,78,79,80,90,91,92,93,94,99,111,120,121,128,142,144,160,170,180,190,200,201,202,203,204,205,206,207,220,221,222,223,224,225,226,227
2,68858,6043392,1,8,2,49,2537.4,16.33,18.37,34.69,32.65,32.65,28,21,38,0,21,0,32,0,0,0,12,22,0,0,11,11,14,0,0,45,17,0,0,0,0,0,11,32,13,0,0,0,0,0,18,0,0,0,0,0,0,0,0,0,0,0
4,68866,6043475,1,8,2,38,2554.9,15.79,18.42,34.21,39.47,26.32,22,16,34,0,0,0,13,0,0,0,0,12,0,12,0,0,0,12,0,27,25,0,0,11,0,0,0,11,23,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,11,0
5,68866,6043483,1,8,2,415,2660.1,59.04,17.11,76.14,15.66,8.19,203,212,406,60,68,0,35,0,0,110,11,46,0,207,0,12,39,155,184,392,380,0,19,23,0,30,0,77,278,0,0,0,0,0,12,0,11,0,0,0,107,0,34,0,196,28
6,68874,6043509,1,8,2,53,2604.0,35.85,16.98,52.83,32.08,15.09,25,28,50,0,0,0,12,0,0,13,0,11,0,21,0,0,13,21,11,49,41,0,0,0,0,0,0,12,33,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,20,0
8,68882,6043525,1,8,2,345,2658.5,59.77,14.87,74.64,14.29,11.08,173,172,334,0,81,0,42,0,0,78,13,43,0,175,0,16,30,107,175,318,303,0,0,27,0,25,11,92,248,0,0,0,0,0,16,0,13,0,0,0,70,0,27,0,162,25
9,68890,6043665,1,8,2,254,2561.6,19.29,23.23,42.52,28.35,29.13,131,123,221,0,84,0,127,0,0,0,0,132,0,98,80,23,32,58,51,239,127,0,0,15,31,11,33,117,133,0,0,0,0,0,112,0,0,0,0,0,0,0,20,0,89,0
11,68908,6043707,1,8,2,176,2669.3,61.93,17.61,79.55,14.20,6.25,87,89,173,22,15,0,0,0,0,55,0,0,0,109,0,0,0,49,112,160,176,0,0,16,0,0,0,18,136,0,0,0,0,0,0,0,0,0,0,0,55,0,0,0,109,0
12,68916,112284,1,8,2,72,2493.3,9.72,8.33,18.06,26.39,55.56,24,48,70,0,12,0,37,0,0,0,0,11,0,35,0,0,0,0,0,60,35,0,64,12,0,0,0,14,52,0,0,0,0,0,0,0,17,0,0,0,0,0,0,0,18,0
14,68916,6043723,1,8,2,214,2523.5,10.28,21.96,32.24,25.70,42.06,96,118,176,24,80,0,124,0,0,26,81,64,0,19,11,43,67,59,0,204,90,0,24,0,36,15,37,117,72,0,0,0,13,36,42,0,17,0,0,0,13,45,22,0,0,0
15,68916,6043756,1,8,2,197,2538.4,13.71,22.84,36.55,29.44,34.01,101,96,140,16,90,0,142,0,0,19,77,88,0,0,16,55,42,46,0,186,55,0,34,11,48,0,54,144,34,0,0,0,18,43,74,0,0,0,0,0,0,34,14,0,0,0


In [19]:
# Set the column data types as following

column_list = df_schools_final.columns[12:58]
df_schools_final[column_list] = df_schools_final[column_list].astype("float")
df_schools_final[["Mean Scale Score"]] = df_schools_final[["Mean Scale Score"]].astype("float")
df_schools_final[["Students Tested"]] = df_schools_final[["Students Tested"]].astype("float")
df_schools_final[["Percentage Standard Exceeded"]] = df_schools_final[["Percentage Standard Exceeded"]].astype("float")
df_schools_final[["Percentage Standard Met"]] = df_schools_final[["Percentage Standard Met"]].astype("float")
df_schools_final[["Percentage Standard Met and Above"]] = df_schools_final[["Percentage Standard Met and Above"]].astype("float")
df_schools_final[["Percentage Standard Nearly Met"]] = df_schools_final[["Percentage Standard Nearly Met"]].astype("float")
df_schools_final[["Percentage Standard Not Met"]] = df_schools_final[["Percentage Standard Not Met"]].astype("float")
df_schools_final[["District Code"]] = df_schools_final[["District Code"]].astype("int")
df_schools_final[["School Code"]] = df_schools_final[["School Code"]].astype("int")
df_schools_final[["Subgroup ID"]] = df_schools_final[["Subgroup ID"]].astype("int")
df_schools_final[["Grade"]] = df_schools_final[["Grade"]].astype("int")
df_schools_final[["Test Id"]] = df_schools_final[["Test Id"]].astype("int")


## Data Normalization

In order to equalize the impact of magnitude differences between various features, normalize the data in varoius subgroup columns, used the following:

* Mean Scale Score: Normalized using Min-Max feature scaling
* All Subgroup counts were expressed as percentage of Students tested

In addition, the SK Learning StandardScaler was applied to normalize as per best practice before K-Means Clustering

In [20]:
# Just keep the following for this clustering analysis and drop all other columns

#3	3	 "Male"	 "Gender"
#4	4	 "Female"	 "Gender"
#31	31	 "Economically disadvantaged"	 "Economic Status"
#74	74	 "Black or African American"	 "Ethnicity"
#75	75	 "American Indian or Alaska Native"	 "Ethnicity"
#76	76	 "Asian"	 "Ethnicity"
#77	77	 "Filipino"	 "Ethnicity"
#78	78	 "Hispanic or Latino"	 "Ethnicity"
#79	79	 "Native Hawaiian or Pacific Islander"	 "Ethnicity"
#80	80	 "White"	 "Ethnicity"
#90	90	 "Not a high school graduate"	 "Parent Education"
#91	91	 "High school graduate"	 "Parent Education"
#92	92	 "Some college (includes AA degree)"	 "Parent Education"
#93	93	 "College graduate"	 "Parent Education"
#94	94	 "Graduate school/Post graduate"	 "Parent Education"

column_list1 = df_schools_final.columns[14:18]
column_list2 = df_schools_final.columns[31:58]

df_schools_final.drop(column_list1, axis = 1, inplace=True)
df_schools_final.drop(column_list2, axis = 1, inplace=True)

df_schools_final.drop('District Code', axis=1, inplace=True)

df_schools_final

,School Code,Subgroup ID,Grade,Test Id,Students Tested,Mean Scale Score,Percentage Standard Exceeded,Percentage Standard Met,Percentage Standard Met and Above,Percentage Standard Nearly Met,Percentage Standard Not Met,3,4,31,74,75,76,77,78,79,80,90,91,92,93,94
2,6043392,1,8,2,49.0,2537.4,16.33,18.37,34.69,32.65,32.65,28.0,21.0,32.0,0.0,0.0,0.0,12.0,22.0,0.0,0.0,11.0,11.0,14.0,0.0,0.0
4,6043475,1,8,2,38.0,2554.9,15.79,18.42,34.21,39.47,26.32,22.0,16.0,13.0,0.0,0.0,0.0,0.0,12.0,0.0,12.0,0.0,0.0,0.0,12.0,0.0
5,6043483,1,8,2,415.0,2660.1,59.04,17.11,76.14,15.66,8.19,203.0,212.0,35.0,0.0,0.0,110.0,11.0,46.0,0.0,207.0,0.0,12.0,39.0,155.0,184.0
6,6043509,1,8,2,53.0,2604.0,35.85,16.98,52.83,32.08,15.09,25.0,28.0,12.0,0.0,0.0,13.0,0.0,11.0,0.0,21.0,0.0,0.0,13.0,21.0,11.0
8,6043525,1,8,2,345.0,2658.5,59.77,14.87,74.64,14.29,11.08,173.0,172.0,42.0,0.0,0.0,78.0,13.0,43.0,0.0,175.0,0.0,16.0,30.0,107.0,175.0
9,6043665,1,8,2,254.0,2561.6,19.29,23.23,42.52,28.35,29.13,131.0,123.0,127.0,0.0,0.0,0.0,0.0,132.0,0.0,98.0,80.0,23.0,32.0,58.0,51.0
11,6043707,1,8,2,176.0,2669.3,61.93,17.61,79.55,14.20,6.25,87.0,89.0,0.0,0.0,0.0,55.0,0.0,0.0,0.0,109.0,0.0,0.0,0.0,49.0,112.0
12,112284,1,8,2,72.0,2493.3,9.72,8.33,18.06,26.39,55.56,24.0,48.0,37.0,0.0,0.0,0.0,0.0,11.0,0.0,35.0,0.0,0.0,0.0,0.0,0.0
14,6043723,1,8,2,214.0,2523.5,10.28,21.96,32.24,25.70,42.06,96.0,118.0,124.0,0.0,0.0,26.0,81.0,64.0,0.0,19.0,11.0,43.0,67.0,59.0,0.0
15,6043756,1,8,2,197.0,2538.4,13.71,22.84,36.55,29.44,34.01,101.0,96.0,142.0,0.0,0.0,19.0,77.0,88.0,0.0,0.0,16.0,55.0,42.0,46.0,0.0


In [21]:
# Using the min-max scaling for the Mean scale score column

df_schools_final['Mean Scale Score']=(df_schools_final['Mean Scale Score']-df_schools_final['Mean Scale Score'].min())/(df_schools_final['Mean Scale Score'].max()-df_schools_final['Mean Scale Score'].min())

df_schools_final.head()

,School Code,Subgroup ID,Grade,Test Id,Students Tested,Mean Scale Score,Percentage Standard Exceeded,Percentage Standard Met,Percentage Standard Met and Above,Percentage Standard Nearly Met,Percentage Standard Not Met,3,4,31,74,75,76,77,78,79,80,90,91,92,93,94
2,6043392,1,8,2,49.0,0.385845,16.33,18.37,34.69,32.65,32.65,28.0,21.0,32.0,0.0,0.0,0.0,12.0,22.0,0.0,0.0,11.0,11.0,14.0,0.0,0.0
4,6043475,1,8,2,38.0,0.452435,15.79,18.42,34.21,39.47,26.32,22.0,16.0,13.0,0.0,0.0,0.0,0.0,12.0,0.0,12.0,0.0,0.0,0.0,12.0,0.0
5,6043483,1,8,2,415.0,0.852740,59.04,17.11,76.14,15.66,8.19,203.0,212.0,35.0,0.0,0.0,110.0,11.0,46.0,0.0,207.0,0.0,12.0,39.0,155.0,184.0
6,6043509,1,8,2,53.0,0.639269,35.85,16.98,52.83,32.08,15.09,25.0,28.0,12.0,0.0,0.0,13.0,0.0,11.0,0.0,21.0,0.0,0.0,13.0,21.0,11.0
8,6043525,1,8,2,345.0,0.846651,59.77,14.87,74.64,14.29,11.08,173.0,172.0,42.0,0.0,0.0,78.0,13.0,43.0,0.0,175.0,0.0,16.0,30.0,107.0,175.0


In [22]:
# For the subgroup columns calculate the percentage based on the Students tested

column_list3 = df_schools_final.columns[11:] # for all subgroups

for i in column_list3:
    df_schools_final[i] = 100*df_schools_final[i]/df_schools_final['Students Tested']
    
df_schools_final

,School Code,Subgroup ID,Grade,Test Id,Students Tested,Mean Scale Score,Percentage Standard Exceeded,Percentage Standard Met,Percentage Standard Met and Above,Percentage Standard Nearly Met,Percentage Standard Not Met,3,4,31,74,75,76,77,78,79,80,90,91,92,93,94
2,6043392,1,8,2,49.0,0.385845,16.33,18.37,34.69,32.65,32.65,57.142857,42.857143,65.306122,0.0,0.0,0.000000,24.489796,44.897959,0.000000,0.000000,22.448980,22.448980,28.571429,0.000000,0.000000
4,6043475,1,8,2,38.0,0.452435,15.79,18.42,34.21,39.47,26.32,57.894737,42.105263,34.210526,0.0,0.0,0.000000,0.000000,31.578947,0.000000,31.578947,0.000000,0.000000,0.000000,31.578947,0.000000
5,6043483,1,8,2,415.0,0.852740,59.04,17.11,76.14,15.66,8.19,48.915663,51.084337,8.433735,0.0,0.0,26.506024,2.650602,11.084337,0.000000,49.879518,0.000000,2.891566,9.397590,37.349398,44.337349
6,6043509,1,8,2,53.0,0.639269,35.85,16.98,52.83,32.08,15.09,47.169811,52.830189,22.641509,0.0,0.0,24.528302,0.000000,20.754717,0.000000,39.622642,0.000000,0.000000,24.528302,39.622642,20.754717
8,6043525,1,8,2,345.0,0.846651,59.77,14.87,74.64,14.29,11.08,50.144928,49.855072,12.173913,0.0,0.0,22.608696,3.768116,12.463768,0.000000,50.724638,0.000000,4.637681,8.695652,31.014493,50.724638
9,6043665,1,8,2,254.0,0.477930,19.29,23.23,42.52,28.35,29.13,51.574803,48.425197,50.000000,0.0,0.0,0.000000,0.000000,51.968504,0.000000,38.582677,31.496063,9.055118,12.598425,22.834646,20.078740
11,6043707,1,8,2,176.0,0.887747,61.93,17.61,79.55,14.20,6.25,49.431818,50.568182,0.000000,0.0,0.0,31.250000,0.000000,0.000000,0.000000,61.931818,0.000000,0.000000,0.000000,27.840909,63.636364
12,112284,1,8,2,72.0,0.218037,9.72,8.33,18.06,26.39,55.56,33.333333,66.666667,51.388889,0.0,0.0,0.000000,0.000000,15.277778,0.000000,48.611111,0.000000,0.000000,0.000000,0.000000,0.000000
14,6043723,1,8,2,214.0,0.332953,10.28,21.96,32.24,25.70,42.06,44.859813,55.140187,57.943925,0.0,0.0,12.149533,37.850467,29.906542,0.000000,8.878505,5.140187,20.093458,31.308411,27.570093,0.000000
15,6043756,1,8,2,197.0,0.389650,13.71,22.84,36.55,29.44,34.01,51.269036,48.730964,72.081218,0.0,0.0,9.644670,39.086294,44.670051,0.000000,0.000000,8.121827,27.918782,21.319797,23.350254,0.000000


In [23]:
# Extract text between the quotes

def getTextInQuotes(text):      
  import re
  matches=re.findall(r'\"(.+?)\"',text)
  return ",".join(matches)

In [24]:
# Map the subgroup ids to descriptive names, use the provided "CAA_ca_Subgroups.csv". 
# If loading of the Subgroup file fails, then leave the columns as is in IDs

try:
    # Read the provided subgroup excel file
    df_schools_subgroups = pd.read_csv('CAA_ca_Subgroups.csv',header = None)
    df_schools_subgroups.head()

    # For readability sake
    df_schools_subgroups.rename(columns={0:'ID1',1:'ID2',2:'Description',3:'Category'}, inplace = True)
    df_schools_subgroups.set_index("ID1", inplace = True)
    df_schools_subgroups.head()

    # Create Column Map Dictionary
    cols_array = df_schools_final.columns.values
    column_map_dict = {}
    for i in range(cols_array.size):
        try:
            col_int = int(cols_array[i]) # if we can convert the column name to a number that is subgroup
            nstr = getTextInQuotes(df_schools_subgroups.loc[int(cols_array[i])].Description)
            column_map_dict[col_int] = nstr
        except:
            column_map_dict[cols_array[i]] = cols_array[i]

    column_map_dict
    
    # Apply the column names mapping of subgroup numbers to descriptive name
    df_schools_final.rename(columns = column_map_dict, inplace = True)
    
except:
    print("Unable to load the Subgroup ids to descriptive names mapping file, therefore, leaving the column headers as is in IDs")

df_schools_final

,School Code,Subgroup ID,Grade,Test Id,Students Tested,Mean Scale Score,Percentage Standard Exceeded,Percentage Standard Met,Percentage Standard Met and Above,Percentage Standard Nearly Met,Percentage Standard Not Met,Male,Female,Economically disadvantaged,Black or African American,American Indian or Alaska Native,Asian,Filipino,Hispanic or Latino,Native Hawaiian or Pacific Islander,White,Not a high school graduate,High school graduate,Some college (includes AA degree),College graduate,Graduate school/Post graduate
2,6043392,1,8,2,49.0,0.385845,16.33,18.37,34.69,32.65,32.65,57.142857,42.857143,65.306122,0.0,0.0,0.000000,24.489796,44.897959,0.000000,0.000000,22.448980,22.448980,28.571429,0.000000,0.000000
4,6043475,1,8,2,38.0,0.452435,15.79,18.42,34.21,39.47,26.32,57.894737,42.105263,34.210526,0.0,0.0,0.000000,0.000000,31.578947,0.000000,31.578947,0.000000,0.000000,0.000000,31.578947,0.000000
5,6043483,1,8,2,415.0,0.852740,59.04,17.11,76.14,15.66,8.19,48.915663,51.084337,8.433735,0.0,0.0,26.506024,2.650602,11.084337,0.000000,49.879518,0.000000,2.891566,9.397590,37.349398,44.337349
6,6043509,1,8,2,53.0,0.639269,35.85,16.98,52.83,32.08,15.09,47.169811,52.830189,22.641509,0.0,0.0,24.528302,0.000000,20.754717,0.000000,39.622642,0.000000,0.000000,24.528302,39.622642,20.754717
8,6043525,1,8,2,345.0,0.846651,59.77,14.87,74.64,14.29,11.08,50.144928,49.855072,12.173913,0.0,0.0,22.608696,3.768116,12.463768,0.000000,50.724638,0.000000,4.637681,8.695652,31.014493,50.724638
9,6043665,1,8,2,254.0,0.477930,19.29,23.23,42.52,28.35,29.13,51.574803,48.425197,50.000000,0.0,0.0,0.000000,0.000000,51.968504,0.000000,38.582677,31.496063,9.055118,12.598425,22.834646,20.078740
11,6043707,1,8,2,176.0,0.887747,61.93,17.61,79.55,14.20,6.25,49.431818,50.568182,0.000000,0.0,0.0,31.250000,0.000000,0.000000,0.000000,61.931818,0.000000,0.000000,0.000000,27.840909,63.636364
12,112284,1,8,2,72.0,0.218037,9.72,8.33,18.06,26.39,55.56,33.333333,66.666667,51.388889,0.0,0.0,0.000000,0.000000,15.277778,0.000000,48.611111,0.000000,0.000000,0.000000,0.000000,0.000000
14,6043723,1,8,2,214.0,0.332953,10.28,21.96,32.24,25.70,42.06,44.859813,55.140187,57.943925,0.0,0.0,12.149533,37.850467,29.906542,0.000000,8.878505,5.140187,20.093458,31.308411,27.570093,0.000000
15,6043756,1,8,2,197.0,0.389650,13.71,22.84,36.55,29.44,34.01,51.269036,48.730964,72.081218,0.0,0.0,9.644670,39.086294,44.670051,0.000000,0.000000,8.121827,27.918782,21.319797,23.350254,0.000000


In [25]:
df_schools_final.shape

(51, 26)

## Analysis <a name="analysis"></a>

Now that we have the selected features (columns) and data normalized, we can feed in to the K-means clustering algorithm. As I mentioned earlier, there may be many different ways (methods) to select various sub groups and, I just picked the following sub group categories and respective sub group Ids, as per my personal interest of, **Gender**, **Economic Status**, **Ethnicity** and **Parent Education**

Other sub-groups that breakdown the Immigration Status, English Language Fluency, Disability status were dropped from analysis at this time to understand impact of selected features. These could be added back per the stakeholders' interests and needs or if the cluster definition is not clear. 

K-Means clustering was applied with 6 Clusters with 22 features (out of 26 columns) and for 51 Schools.

### Let's start clustering

In [26]:
import random # library for random number generation
import numpy as np # library for vectorized computation
import pandas as pd # library to process data as dataframes

import matplotlib.pyplot as plt # plotting library
# backend for rendering plots within the browser
%matplotlib inline 

from sklearn.cluster import KMeans 
from sklearn.preprocessing import StandardScaler

print('Libraries imported.')

Libraries imported.


In [27]:
X = df_schools_final.values[:,4:] # Using all features retained
X = np.nan_to_num(X)
cluster_dataset = StandardScaler().fit_transform(X)
cluster_dataset

array([[-0.86146974, -0.45675369, -0.66964659, ...,  1.47104402,
        -1.22568988, -0.86277312],
       [-0.96660016, -0.2111502 , -0.69538965, ..., -0.91109279,
         0.72760163, -0.86277312],
       [ 2.63650595,  1.26527762,  1.36643837, ..., -0.12757068,
         1.08452839,  0.99279624],
       ...,
       [ 0.37142333, -1.09391931, -0.653438  , ...,  1.00933773,
        -0.63494722, -0.86277312],
       [ 0.71548651,  0.73617753,  0.73478007, ...,  1.97196064,
         1.2600409 , -0.60853712],
       [-0.98571478,  1.25966382,  1.33259102, ..., -0.91109279,
         0.66430052,  1.69479757]])

In [28]:
# set number of clusters
kclusters = 6

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(cluster_dataset)

labels = kmeans.labels_

print(labels)

[3 3 2 3 2 3 2 4 3 3 3 1 3 0 0 0 4 0 0 2 0 1 1 1 1 5 5 1 4 3 0 1 1 3 1 1 1
 0 3 0 0 0 3 3 3 2 0 3 1 3 0]


In [29]:
from collections import Counter
Counter(labels)

Counter({3: 16, 2: 5, 4: 3, 1: 12, 0: 13, 5: 2})

In [30]:
# Assign Cluster Labels to the data

df_schools_final["Cluster_Label"] = labels

df_schools_final.head(5)

,School Code,Subgroup ID,Grade,Test Id,Students Tested,Mean Scale Score,Percentage Standard Exceeded,Percentage Standard Met,Percentage Standard Met and Above,Percentage Standard Nearly Met,Percentage Standard Not Met,Male,Female,Economically disadvantaged,Black or African American,American Indian or Alaska Native,Asian,Filipino,Hispanic or Latino,Native Hawaiian or Pacific Islander,White,Not a high school graduate,High school graduate,Some college (includes AA degree),College graduate,Graduate school/Post graduate,Cluster_Label
2,6043392,1,8,2,49.0,0.385845,16.33,18.37,34.69,32.65,32.65,57.142857,42.857143,65.306122,0.0,0.0,0.000000,24.489796,44.897959,0.0,0.000000,22.44898,22.448980,28.571429,0.000000,0.000000,3
4,6043475,1,8,2,38.0,0.452435,15.79,18.42,34.21,39.47,26.32,57.894737,42.105263,34.210526,0.0,0.0,0.000000,0.000000,31.578947,0.0,31.578947,0.00000,0.000000,0.000000,31.578947,0.000000,3
5,6043483,1,8,2,415.0,0.852740,59.04,17.11,76.14,15.66,8.19,48.915663,51.084337,8.433735,0.0,0.0,26.506024,2.650602,11.084337,0.0,49.879518,0.00000,2.891566,9.397590,37.349398,44.337349,2
6,6043509,1,8,2,53.0,0.639269,35.85,16.98,52.83,32.08,15.09,47.169811,52.830189,22.641509,0.0,0.0,24.528302,0.000000,20.754717,0.0,39.622642,0.00000,0.000000,24.528302,39.622642,20.754717,3
8,6043525,1,8,2,345.0,0.846651,59.77,14.87,74.64,14.29,11.08,50.144928,49.855072,12.173913,0.0,0.0,22.608696,3.768116,12.463768,0.0,50.724638,0.00000,4.637681,8.695652,31.014493,50.724638,2


In [31]:
# Merge the Schools entity data with schools cluster data to visualize on Folium map
df_schools_cluster = pd.merge(df_schools_final, df_schools_sm, on='School Code')

df_schools_cluster.head(10)

,School Code,Subgroup ID,Grade,Test Id,Students Tested,Mean Scale Score,Percentage Standard Exceeded,Percentage Standard Met,Percentage Standard Met and Above,Percentage Standard Nearly Met,Percentage Standard Not Met,Male,Female,Economically disadvantaged,Black or African American,American Indian or Alaska Native,Asian,Filipino,Hispanic or Latino,Native Hawaiian or Pacific Islander,White,Not a high school graduate,High school graduate,Some college (includes AA degree),College graduate,Graduate school/Post graduate,Cluster_Label,County Code,District Code,Test Year,Type Id,County Name,District Name,School Name,Zip Code,Latitude,Longitude
0,6043392,1,8,2,49.0,0.385845,16.33,18.37,34.69,32.65,32.65,57.142857,42.857143,65.306122,0.0,0.0,0.000000,24.489796,44.897959,0.0,0.000000,22.448980,22.448980,28.571429,0.000000,0.000000,3,41,68858,2018,7,San Mateo,Bayshore Elementary,Bayshore,94014,37.706321,-122.413305
1,6043475,1,8,2,38.0,0.452435,15.79,18.42,34.21,39.47,26.32,57.894737,42.105263,34.210526,0.0,0.0,0.000000,0.000000,31.578947,0.0,31.578947,0.000000,0.000000,0.000000,31.578947,0.000000,3,41,68866,2018,7,San Mateo,Belmont-Redwood Shores Elementary,Nesbit Elementary,94002,37.524978,-122.274819
2,6043483,1,8,2,415.0,0.852740,59.04,17.11,76.14,15.66,8.19,48.915663,51.084337,8.433735,0.0,0.0,26.506024,2.650602,11.084337,0.0,49.879518,0.000000,2.891566,9.397590,37.349398,44.337349,2,41,68866,2018,7,San Mateo,Belmont-Redwood Shores Elementary,Ralston Intermediate,94002,37.511340,-122.311995
3,6043509,1,8,2,53.0,0.639269,35.85,16.98,52.83,32.08,15.09,47.169811,52.830189,22.641509,0.0,0.0,24.528302,0.000000,20.754717,0.0,39.622642,0.000000,0.000000,24.528302,39.622642,20.754717,3,41,68874,2018,7,San Mateo,Brisbane Elementary,Lipman Middle,94005,37.685971,-122.408057
4,6043525,1,8,2,345.0,0.846651,59.77,14.87,74.64,14.29,11.08,50.144928,49.855072,12.173913,0.0,0.0,22.608696,3.768116,12.463768,0.0,50.724638,0.000000,4.637681,8.695652,31.014493,50.724638,2,41,68882,2018,7,San Mateo,Burlingame Elementary,Burlingame Intermediate,94010,37.588453,-122.386318
5,6043665,1,8,2,254.0,0.477930,19.29,23.23,42.52,28.35,29.13,51.574803,48.425197,50.000000,0.0,0.0,0.000000,0.000000,51.968504,0.0,38.582677,31.496063,9.055118,12.598425,22.834646,20.078740,3,41,68890,2018,7,San Mateo,Cabrillo Unified,Manuel F. Cunha Intermediate,94019,37.462455,-122.431900
6,6043707,1,8,2,176.0,0.887747,61.93,17.61,79.55,14.20,6.25,49.431818,50.568182,0.000000,0.0,0.0,31.250000,0.000000,0.000000,0.0,61.931818,0.000000,0.000000,0.000000,27.840909,63.636364,2,41,68908,2018,7,San Mateo,Hillsborough City Elementary,Crocker Middle,94010,37.565868,-122.363694
7,112284,1,8,2,72.0,0.218037,9.72,8.33,18.06,26.39,55.56,33.333333,66.666667,51.388889,0.0,0.0,0.000000,0.000000,15.277778,0.0,48.611111,0.000000,0.000000,0.000000,0.000000,0.000000,4,41,68916,2018,9,San Mateo,California Virtual Academy San Mateo,California Virtual Academy San Mateo,93065,34.275812,-118.797952
8,6043723,1,8,2,214.0,0.332953,10.28,21.96,32.24,25.70,42.06,44.859813,55.140187,57.943925,0.0,0.0,12.149533,37.850467,29.906542,0.0,8.878505,5.140187,20.093458,31.308411,27.570093,0.000000,3,41,68916,2018,7,San Mateo,Jefferson Elementary,Benjamin Franklin Intermediate,94015,37.695055,-122.482228
9,6043756,1,8,2,197.0,0.389650,13.71,22.84,36.55,29.44,34.01,51.269036,48.730964,72.081218,0.0,0.0,9.644670,39.086294,44.670051,0.0,0.000000,8.121827,27.918782,21.319797,23.350254,0.000000,3,41,68916,2018,7,San Mateo,Jefferson Elementary,Thomas R. Pollicita Middle,94014,37.690474,-122.459108


In [32]:
df_schools_cluster=df_schools_cluster[["County Code","County Name","District Code","District Name","School Code","School Name","Cluster_Label","Latitude","Longitude","Zip Code"]]

In [33]:
df_schools_cluster

,County Code,County Name,District Code,District Name,School Code,School Name,Cluster_Label,Latitude,Longitude,Zip Code
0,41,San Mateo,68858,Bayshore Elementary,6043392,Bayshore,3,37.706321,-122.413305,94014
1,41,San Mateo,68866,Belmont-Redwood Shores Elementary,6043475,Nesbit Elementary,3,37.524978,-122.274819,94002
2,41,San Mateo,68866,Belmont-Redwood Shores Elementary,6043483,Ralston Intermediate,2,37.511340,-122.311995,94002
3,41,San Mateo,68874,Brisbane Elementary,6043509,Lipman Middle,3,37.685971,-122.408057,94005
4,41,San Mateo,68882,Burlingame Elementary,6043525,Burlingame Intermediate,2,37.588453,-122.386318,94010
5,41,San Mateo,68890,Cabrillo Unified,6043665,Manuel F. Cunha Intermediate,3,37.462455,-122.431900,94019
6,41,San Mateo,68908,Hillsborough City Elementary,6043707,Crocker Middle,2,37.565868,-122.363694,94010
7,41,San Mateo,68916,California Virtual Academy San Mateo,112284,California Virtual Academy San Mateo,4,34.275812,-118.797952,93065
8,41,San Mateo,68916,Jefferson Elementary,6043723,Benjamin Franklin Intermediate,3,37.695055,-122.482228,94015
9,41,San Mateo,68916,Jefferson Elementary,6043756,Thomas R. Pollicita Middle,3,37.690474,-122.459108,94014


In [34]:
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import matplotlib.cm as cm
import matplotlib.colors as colors
print('Libraries imported.')

Libraries imported.


In [35]:
# Use geopy library if using Nominatim, to get the latitude and longitude values of San Mateo, CA.
# If using Nominatim, define an instance of the geocoder, we need to define a user_agent.
# We can either use Nominatim or Google Map API

address = 'San Mateo, CA'

# Using Nominatim
geolocator = Nominatim(user_agent="sm_explorer")
location = geolocator.geocode(address)
sm_latitude = location.latitude
sm_longitude = location.longitude

print('The geograpical coordinate of {} are {}, {}'.format(address, sm_latitude, sm_longitude))

The geograpical coordinate of San Mateo, CA are 37.496904, -122.3330573


In [36]:
# create map
map_clusters = folium.Map(location=[sm_latitude, sm_longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []

for lat, lon, school, cluster in zip(df_schools_cluster['Latitude'], df_schools_cluster['Longitude'], df_schools_cluster['School Name'], df_schools_cluster['Cluster_Label']):
    
    color = fill_color = rainbow[cluster-1]

    label_text = str(school) + ' Cluster ' + str(cluster)
 
    label = folium.Popup(label_text, parse_html=True)

    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=color,
        fill=True,
        fill_color=fill_color,
        fill_opacity=0.7).add_to(map_clusters)
    

map_clusters

## Results and Discussion <a name="results"></a>

In [37]:
# Clusters

df_schools_final.iloc[:,1:].groupby('Cluster_Label').mean() # ignore the School Code

,Subgroup ID,Grade,Test Id,Students Tested,Mean Scale Score,Percentage Standard Exceeded,Percentage Standard Met,Percentage Standard Met and Above,Percentage Standard Nearly Met,Percentage Standard Not Met,Male,Female,Economically disadvantaged,Black or African American,American Indian or Alaska Native,Asian,Filipino,Hispanic or Latino,Native Hawaiian or Pacific Islander,White,Not a high school graduate,High school graduate,Some college (includes AA degree),College graduate,Graduate school/Post graduate
Cluster_Label,,,,,,,,,,,,,,,,,,,,,,,,,
0,1,8,2,103.384615,0.813195,55.035385,20.246923,75.282308,14.392308,10.325385,48.358287,51.641713,4.984733,0.0,0.0,3.793298,0.000000,16.195104,0.000000,60.078769,0.000000,0.303644,7.035667,30.595895,49.940929
1,1,8,2,74.166667,0.252220,10.382500,14.244167,24.625833,23.682500,51.689167,57.113831,42.886169,81.812417,0.0,0.0,0.000000,2.477477,84.153528,0.000000,0.000000,45.754362,28.091962,4.585864,0.795880,0.000000
2,1,8,2,311.600000,0.816895,55.608000,17.966000,73.576000,15.702000,10.722000,49.061890,50.938110,9.914763,0.0,0.0,34.033261,1.283744,9.515803,0.000000,43.628331,0.000000,4.434421,7.659166,35.220492,43.712519
3,1,8,2,192.000000,0.468750,24.241875,18.681875,42.923750,25.342500,31.733125,51.221786,48.778214,44.146357,0.0,0.0,10.224831,13.105930,39.024300,0.960648,19.122415,10.215525,15.941394,23.282605,26.700322,11.474023
4,1,8,2,38.333333,0.222476,12.693333,9.126667,21.823333,15.076667,63.110000,11.111111,65.512266,61.934824,0.0,0.0,0.000000,0.000000,24.140212,0.000000,40.446128,0.000000,0.000000,0.000000,0.000000,0.000000
5,1,8,2,58.500000,0.071918,2.590000,6.050000,8.640000,20.745000,70.610000,59.210526,40.789474,94.956140,0.0,0.0,0.000000,0.000000,68.421053,20.482456,0.000000,52.280702,28.070175,0.000000,0.000000,0.000000


In [38]:
# Cluster 0: High performing, very well doing economically, White majority followed by Hispanic and Asian, with highly educated parents
#
# Cluster 1: Low performing, highly Economic disadvantaged, Hispanic or Latino majority, with highly uneducated parents
#
# Cluster 2: High performing, very well doing economically, White majority followed by Asian with highly educated parents
#
# Cluster 3: Medium performing, economically disadvantaged, Hispanic majority followed by White, Filipino and Asian with college or equivalent educated parents
#
# Cluster 4: Low performing, economically disadvantaged, high female populated, White majority followed by Hispanic
#
# Cluster 5: Poorly performing, high male populated, very highly economically disadvantaged, Hispanic majority followed by Native Hawaiian or Pacific Islanders with highly uneducated parents

It is interesting to see clustering of San Mateo county schools based on the 8th grade tests performance and the selected subgroup features of the data, looks like the Ethnicity and Parents education background has influenced these clusters.

* Cluster 0 (Red): High performing, very well doing economically, White majority followed by Hispanic and Asian, with highly educated parents

* Cluster 1 (Indigo): Low performing, highly Economic disadvantaged, Hispanic or Latino majority, with highly uneducated parents

* Cluster 2 (Blue): High performing, very well doing economically, White majority followed by Asian with highly educated parents

* Cluster 3 (Turquoise): Medium performing, economically disadvantaged, Hispanic majority followed by White, Filipino and Asian with college or equivalent educated parents

* Cluster 4 (Madang): Low performing, economically disadvantaged, high female populated, White majority followed by Hispanic

* Cluster 5 (Sunshade): Poorly performing, high male populated, very highly economically disadvantaged, Hispanic majority followed by Native Hawaiian or Pacific Islanders with highly uneducated parents

## Conclusion <a name="conclusion"></a>

The purpose of study was to help in making certain decisions based on individual’s choice, such as moving to San Mateo county in California, for either to be part of their respective ethnic culture, easier commute or may have more affordable housing or for any other personal and/or business reasons. This project in particular does the clustering of the schools in San Mateo for the 8th grade, the factors that influenced this clustering most seem to be Mean Scale Score, Ethnicity, Economic status and Parent's educational level which may all be inter-related.

There are many possible combination of features (Subgroups) can be pursued based on individuals' preferences or interests, such as:
* Immigration status as features and do the exercise again
* English Language Arts (ELA) scores and compare the clusters with Math clusters
* Different grade levels such as 5th or 11th and compare clusters
* Commute distances for various schools from a certain point
* For home buyers, analyze with home prices for the zip-code and find attractive neighborhoods for their choice.


### Thank You!